# Importing Libraries

In [11]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [12]:
fashion = keras.datasets.fashion_mnist

In [13]:
(train_images,train_labels),(test_images,test_lables) = fashion.load_data()

4422102/4422102 [==============================] - 9s 2us/step


In [14]:
train_images = train_images/255.0

In [15]:
test_images = test_images/255.0

In [16]:
train_images[0].shape

(28, 28)

In [23]:
from keras.preprocessing.image import array_to_img
array_to_img(train_images[10])

In [17]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [39]:
def build_model(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters=hp.Int('conv_1_filter',min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_1_kernal',values=[3,5]),
            activation='relu',
            input_shape=(28,28,1)
        ),
        keras.layers.Conv2D(
        filters = hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
        kernel_size=hp.Choice('conv_2_kernal',values=[3,5]),
        activation='relu'
            ,),
        keras.layers.Flatten(),
        keras.layers.Dense(
        units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
        activation='relu'
        ),
    ])
    
    model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate',
                                                             values=[1e-2,1e-3])),
                 loss="sparse_categorical_crossentropy",
                 metrics = ["accuracy"])
    return model

In [42]:
from keras_tuner import RandomSearch
from keras_tuner.engine import hyperparameters

In [43]:
tuner_search = RandomSearch(build_model,
                           objective="val_accuracy",
                           max_trials=5,directory='output',
                           project_name="Mnist Fashion")

In [44]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 08m 16s]
val_accuracy: 0.10499999672174454

Best val_accuracy So Far: 0.21449999511241913
Total elapsed time: 00h 39m 20s


In [45]:
model=tuner_search.get_best_models(num_models=1)[0]

In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        38448     
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 112)               2602096   
                                                                 
Total params: 2640864 (10.07 MB)
Trainable params: 2640864 (10.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [47]:
model.fit(train_images, train_labels, epochs=5, validation_split=0.1, initial_epoch=3)

Epoch 4/5
1688/1688 [==============================] - 130s 76ms/step - loss: 11.0789 - accuracy: 0.2042 - val_loss: 10.8610 - val_accuracy: 0.2145
Epoch 5/5
1688/1688 [==============================] - 134s 79ms/step - loss: 11.0789 - accuracy: 0.2042 - val_loss: 10.8610 - val_accuracy: 0.2145
